# Лабораторная работа №1

Требования:
* Python >= 3.X
* NLTK >= 3.2.5

Лабораторную работу необходимо выполнять в данном шаблоне. Результат работы выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Для выполнения задания необходимо подобрать корпус текстов (художественных) на английском языке. Объем корпуса должен составлять не менее $3 \cdot 10^7$ символов. Далее все действия будут выполнятся над выбранными данными. Для отладки алгоритмов лучше использовать не весь корпус, а лишь его часть.

In [3]:
import os
from functools import reduce
import nltk
from nltk import RegexpTokenizer
from nltk import word_tokenize
nltk.download('punkt')
# import sqlite3
import random
import requests
from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to /home/yaroslav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
books = (
    "https://www.gutenberg.org/cache/epub/66564/pg66564.txt",
    "https://www.gutenberg.org/cache/epub/66568/pg66568.txt",
    "https://www.gutenberg.org/files/84/84-0.txt",
    "https://www.gutenberg.org/files/1342/1342-0.txt",
    "https://www.gutenberg.org/cache/epub/25344/pg25344.txt",
    "https://www.gutenberg.org/files/2701/2701-0.txt",
    "https://www.gutenberg.org/files/345/345-0.txt",
    "https://www.gutenberg.org/files/174/174-0.txt",
    "https://www.gutenberg.org/files/1232/1232-0.txt",
    "https://www.gutenberg.org/files/16328/16328-0.txt",
    "https://www.gutenberg.org/files/5200/5200-0.txt",
    "https://www.gutenberg.org/files/64317/64317-0.txt",
    "https://www.gutenberg.org/files/2591/2591-0.txt",
    "https://www.gutenberg.org/files/3207/3207-0.txt",
    "https://www.gutenberg.org/files/45/45-0.txt",
    "https://www.gutenberg.org/files/203/203-0.txt",
    "https://www.gutenberg.org/files/42884/42884-0.txt"
)
source = ""
for book in tqdm(books):
    response = requests.get(book)
    if response.ok:
        source = source + response.content.decode("utf-8-sig")
len(sourcece)        

  0%|          | 0/17 [00:00<?, ?it/s]

10436875

In [9]:
conn = sqlite3.connect("wikibooks.sqlite")
raw_text = ""
for row in conn.execute("""select body_text from en"""):
    raw_text += row[0]
    if len(raw_text) > int(3e7):
        break
conn.commit()
conn.close()
assert len(raw_text) > int(3e7), "недостаточно книг"

#### Шаг №1
Заменить все числа, которые представлены цифрами, их текстовым представлением (т.е. прописью). (1 = one, 23 = twenty three, 1042 = one thousand forty two, и т.п.). Методы библиотек не использовать, алгоритм необходимо написать самостоятельно.

In [7]:
encoder = {
    0:"zero",
    1:"one",
    2:"two",
    3:"three",
    4:"four",
    5:"five",
    6:"six",
    7:"seven",
    8:"eight",
    9:"nine",
    10:"ten",
    11:"eleven",
    12:"twelve",
    13:"thirteen",
    14:"fourteen",
    15:"fifteen",
    16:"sixteen",
    17:"seventeen",
    18:"eigthteen",
    19:"nineteen",
    20:"twenty",
    30:"thirty",
    40:"fourty",
    50:"fifty",
    60:"sixty",
    70:"seventy",
    80:"eighty",
    90:"ninety"
}
def converter(num:str)->str:
    """
    Args:
        num(str/int): input numeric
    Return:
        name in english for numeric
    """
    
    try:
        num_int = int(num)
        if num_int <= 20:
            return encoder[num_int]
        elif num_int < 100:
            decil, number = divmod(num_int,10)
            if number:
                return "{} {}".format(encoder[decil*10],encoder[number%10])
            else:
                return "{}".format(encoder[num_int])
        else:
            num = str(num)
            num_lentgh = len(num)
            if num_lentgh == 3:
                if num_int // 100 and num_int % 100:
                    return "{} hundred {}".format(converter(num[0]), converter(num[1:]))
                else:
                    return "{} hundred".format(converter(num[0]))
            if num_lentgh == 4:
                if num_int // 1000 and num_int % 100:
                    return "{} thousand {}".format(converter(num[0]), converter(num[1:]))
                else:
                    return "{} thousand".format(converter(num[0]))   
    except Exception as e:
        return e
    
### Тестирование
sample = random.sample(range(0, 1500), k=15)
dict(zip(sample, list(map(converter, sample))))

{782: 'seven hundred eighty two',
 753: 'seven hundred fifty three',
 733: 'seven hundred thirty three',
 670: 'six hundred seventy',
 1097: 'one thousand ninety seven',
 1387: 'one thousand three hundred eighty seven',
 1048: 'one thousand fourty eight',
 600: 'six hundred',
 581: 'five hundred eighty one',
 532: 'five hundred thirty two',
 625: 'six hundred twenty five',
 1255: 'one thousand two hundred fifty five',
 423: 'four hundred twenty three',
 986: 'nine hundred eighty six',
 1109: 'one thousand one hundred nine'}

#### Шаг №2
Преобразовать текст к нижнему или верхнему регистру.

In [12]:
def upperchar(char:str)->str:
    """ Upper char """

    try:
        char_ord = ord(char)
        if char_ord >= 97:
            char = chr(char_ord-32)
            return char
        else:
            return char
    except:
        return char

def upperstring(text:str)->str:
    """ Upper string """

    text = reduce(lambda x,y: x+y, map(upperchar,text))
    return text

upperstring(raw_text[50000:60000])

"\n4K\n\nROCKET ON LASER BEAM, AND MOON\n\n3764\n\n3687\n\n3722\n\n3610 (A1730)\n\n3637 (DYE)\n\n3699 (1373)\n\n3500\n\n25 1969.06.25 (IVAN KOTLIAREVSKY)[EDIT\x80\\ EDIT SOURCE]\n\n\n\n\n25 BIRTH BICENTENARY OF IVAN KOTLIAREVSKY, UKRAINIAN WRITER (1769-1838) (IVAN KOTLIAREVSKY)\n25. 200-ЛЕТИЕ СО ДНЯ РОЖДЕНИЯ УКРАИНСКОГО ПИСАТЕЛЯ ϸВАНА ϿЕТРОВИЧА ϺОТЛЯРЕВСКОГО (1769-1838)\n\n\n\n#\n\nVOL.\n\nSHORT DESCRIPTION\n\nCPA\n\nSC\n\nLIAPINE\n\nSCOTT\n\nMICHEL\n\nSG\n\nYVERT\n\n\n#1969.46-25.1 C42\x80X\x8030\x80MM\n\n4K\n\nIVAN KOTLIAREVSKY\n\n3765\n\n3688\n\n3723\n\n3611 (A1731)\n\n3638 (DYF)\n\n3700 (1374)\n\n3498\n\n26 1969.06.25 (ICE HOCKEY)[EDIT\x80\\ EDIT SOURCE]\n\n\n\n\n26 SOVIET VICTORY IN THE ICE HOCKEY WORLD CHAMPIONSHIPS, STOCKHOLM, SWEDEN, 1969 (ICE HOCKEY)\n26. ЁОВЕТСКИЕ ХОККЕИСТЫ - ЧЕМПИОНЫ МИРА И ϵВРОПЫ\n\n\n\n#\n\nVOL.\n\nSHORT DESCRIPTION\n\nCPA\n\nSC\n\nLIAPINE\n\nSCOTT\n\nMICHEL\n\nSG\n\nYVERT\n\n\n#1969.47-26.1 C40\x80X\x8028\x80MM\n\n6K\n\n2835 OVERPRINTED ЁТОКГОЛЬМ. 1969\n\

#### Шаг №3
Произвести токенезацию (токенами выступают слова), не учитывая знаки препинания (т.е. необходимо разбить текст на токены (лексемы)):
* С помощью nltk.tokenize.RegexpTokenizer (тут необходимо вспомнить регулярные выражения [posix](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F)).

In [21]:
sample = raw_text[100_000:200_000]
# sample
tokenizer_regexp = RegexpTokenizer("[:punct:]*")
print(f"regexp tokenizer: {tokenizer_regexp.tokenize(sample)}")


regexp tokenizer: ['', '', '', '', '', '', '', '', '', '', '', 't', '', '', '', '', '', 'n', '', 't', '', '', '', '', 'n', '', 'p', '', '', '', '', '', '', '', '', 't', '', '', '', '', '', '', 'nt', '', 'n', '', '', '', '', '', 'n', '', '', 'un', '', '', '', '', 't', '', 'n', '', '', 't', '', '', '', '', '', '', '', '', 'c', '', '', '', '', '', 'n', '', '', '', '', '', '', '', 't', '', '', 't', '', '', '', '', 'c', '', '', 'p', '', '', '', '', '', '', '', 't', '', '', 't', '', '', '', '', 'n', '', '', '', '', '', '', '', '', '', '', '', 't', '', '', '', '', '', '', 'u', '', '', '', '', 'n', '', '', 't', '', '', '', '', 'u', '', '', 't', '', '', '', '', '', '', '', 't', '', '', '', '', '', '', '', '', '', '', '', '', '', 't', '', '', '', '', '', 'u', '', '', 't', '', '', 't', '', '', '', 'p', '', 'cu', '', '', '', '', '', 't', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'cu', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'n', '',

* С помощью nltk.word_tokenize.

In [20]:
print(f"word tokenizer: {word_tokenize(sample)}")

word tokenizer: [']', '.', 'So', 'far', 'it', 'has', 'not', 'been', 'possible', 'to', 'disentangle', 'and', 'understand', 'the', 'odor', 'code', 'in', 'a', 'way', 'that', 'is', 'comparable', 'to', 'the', 'knowledge', 'of', 'the', 'visual', 'and', 'the', 'auditory', 'system', '.', 'This', 'might', 'be', 'due', 'to', 'the', 'peculiarities', 'of', 'smell', 'discussed', 'above', '.', 'There', 'is', 'no', 'easily', 'mappable', 'topographic', 'organization', 'of', 'neurons', 'as', 'the', 'odor', 'space', 'is', 'multidimensional', 'and', 'not', 'continuous', '.', 'Glomerular', 'activity', 'patterns', 'are', 'linked', 'in', 'the', 'mushroom', 'bodies', 'to', 'behavior', '.', 'Most', 'insects', 'show', 'a', 'high', 'amount', 'of', 'plasticity', 'there', 'and', 'e.g', '.', 'bees', 'are', 'able', 'to', 'associate', 'odors', 'with', 'a', 'food', 'reward', 'after', 'only', 'a', 'few', 'presentations', '.', 'Odor', 'Information', 'in', 'the', 'mushroom', 'bodies', 'is', 'said', 'to', 'be', 'represen

#### Шаг №4
Произвести частотный анализ слов. Методы библиотек не использовать, т.е. алгоритм необходимо написать самостоятельно. Cохранить результат в выходной csv файл в порядке убывания частот. Формат выходного файла:

| Word | Count |
| --- | --- |
| cat | 1000 |
| dog | 300 |
| .... | .... |
| butterfly | 34 |

In [22]:
def Counter(text:str, sep=" ")->dict:
    words = text.split(sep)
    unqique_words = set(words)
    pair = {}
    for word in unqique_words:
        pair[word] = words.count(word)
        
    return pair
    
Counter(sample)

{'': 526,
 'bed."': 1,
 'tread': 3,
 'actuator': 6,
 'rather': 7,
 'movies.': 1,
 '20,': 1,
 'execution,': 1,
 'directions': 1,
 'Australia.': 1,
 'values': 7,
 'Him.\nWhen': 1,
 'played': 1,
 'teachings': 1,
 'Blocks\xa0·': 1,
 '\nYour': 1,
 'presses': 2,
 'longer': 3,
 'polished': 1,
 'inhibition': 1,
 'contents[edit\xa0|': 1,
 "they're": 2,
 'screen': 1,
 'used.': 1,
 'oriented': 1,
 'humanity': 4,
 'reliance': 1,
 'parameters.\nBootable': 1,
 'cultures.': 1,
 'serviced': 1,
 'cross-section': 2,
 'receive': 1,
 'shows': 2,
 'Hannah': 1,
 'riding': 1,
 'Wheel\nOne': 1,
 'mouth': 1,
 'Protocol):': 7,
 'United': 1,
 'smaller,': 1,
 'communal': 1,
 'displays.': 1,
 'him,': 1,
 'parasitoids.': 1,
 'behaviour': 1,
 'secrete': 1,
 'hdparm': 1,
 '(2002):': 1,
 'transferred': 1,
 'trail': 3,
 '"Grand': 1,
 'relation,': 1,
 'Ctrl+M,': 1,
 'Reading': 7,
 'palatable.': 1,
 '"God': 1,
 'dynamically': 1,
 'wheels': 5,
 'Biographical': 1,
 'suggests,': 1,
 'reflects': 1,
 'powerful,': 1,
 'me,"': 

#### Шаг №5
Произвести частотный анализ [N-грам](https://en.wikipedia.org/wiki/N-gram) и сохранить в выходной файл в порядке убывания частот (формат файла аналогичен предыдущему примеру, за исключением столбца Word, который заменяется на N-gram. В этой задаче можно использовать готовые библиотеки (смотрим состав библиотеки NLTK). Формат выходного файла:

| N-gram | Count |
| --- | --- |
| cat is | 1000 |
| is a | 300 |
| .... | .... |
| the fish | 34 |

* Биграм

In [35]:
from nltk import ngrams

In [39]:
ngrams = lambda input_list, n: list(zip(*[input_list[i:] for i in range(n)]))
ngrams(['all', 'this', 'happened', 'more', 'or', 'less'], 2)

[('all', 'this'),
 ('this', 'happened'),
 ('happened', 'more'),
 ('more', 'or'),
 ('or', 'less')]

* 3-грам

In [38]:
list(ngrams(['all', 'this', 'happened', 'more', 'or', 'less'], 3))

[('all', 'this', 'happened'),
 ('this', 'happened', 'more'),
 ('happened', 'more', 'or'),
 ('more', 'or', 'less')]

#### Шаг №6 (аналогично шагу 3)
Произвести токенезацию, но уже учитывая знаки препинания:
* С помощью nltk.tokenize.RegexpTokenizer.

In [ ]:
# вставьте код здесь

* С помощью nltk.word_tokenize.

In [ ]:
# вставьте код здесь

#### Шаг №7 (аналогично шагу 4)
Произвести частотный анализ слов.

In [ ]:
# вставьте код здесь

#### Шаг №8 (аналогично шагу 5)
Произвести частотный анализ N-грам.

* Биграм

In [ ]:
# вставьте код здесь

* 3-грам

In [ ]:
# вставьте код здесь